In [367]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-dataset/sample_submission.csv
/kaggle/input/spaceship-dataset/train.csv
/kaggle/input/spaceship-dataset/test.csv


In [368]:
import pandas as pd

In [369]:
space_train= pd.read_csv("/kaggle/input/spaceship-dataset/train.csv")
space_test= pd.read_csv("/kaggle/input/spaceship-dataset/test.csv")
test_ids=space_test['PassengerId']

In [370]:
space_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [371]:
space_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [372]:
space_train.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [373]:
space_train['HomePlanet'].value_counts()

HomePlanet
Earth     4602
Europa    2131
Mars      1759
Name: count, dtype: int64

In [374]:
space_train['Destination'].value_counts()

Destination
TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: count, dtype: int64

In [319]:
space_train['Cabin'].value_counts()

Cabin
G/734/S     8
B/11/S      7
F/1411/P    7
B/82/S      7
G/981/S     7
           ..
G/543/S     1
B/106/P     1
G/542/S     1
F/700/P     1
G/559/P     1
Name: count, Length: 6560, dtype: int64

In [375]:
space_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [376]:
def clean(data):
    data=data.drop(['PassengerId','Name'],axis=1)
    cols=['Age','RoomService','FoodCourt','Spa','ShoppingMall','VRDeck']
    for col in cols:
        data[col].fillna(data[col].median(),inplace=True)
    return data

In [377]:
def split_cabin_column(df):
    # Split the 'Cabin' column into three separate columns
    df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
    # Drop the original 'Cabin' column
    df.drop(columns=['Cabin'], inplace=True)
    return df

In [323]:
# space_train.dropna(inplace=True)

In [378]:
space_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [379]:
space_train=clean(space_train)

/tmp/ipykernel_34/3621272143.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(),inplace=True)
/tmp/ipykernel_34/3621272143.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [380]:
space_train.isnull().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [327]:
space_train['HomePlanet'].mode().values

array(['Earth'], dtype=object)

In [381]:
space_test=clean(space_test)

/tmp/ipykernel_34/3621272143.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(),inplace=True)
/tmp/ipykernel_34/3621272143.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [382]:
space_test.isnull().sum()

HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age               0
VIP              93
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
dtype: int64

In [383]:
space_train.isnull().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [330]:
space_train[space_train['VIP']==True]

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
108,Europa,False,B/1/S,55 Cancri e,48.0,True,0.0,2537.0,87.0,17.0,13.0,True
120,Mars,False,D/3/S,TRAPPIST-1e,61.0,True,2353.0,334.0,9.0,316.0,2.0,False
214,Mars,False,F/42/S,TRAPPIST-1e,32.0,True,181.0,0.0,5.0,1634.0,0.0,False
291,NaN,False,F/61/S,TRAPPIST-1e,59.0,True,1018.0,0.0,209.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8579,Europa,True,B/298/P,55 Cancri e,30.0,True,0.0,0.0,0.0,0.0,0.0,True
8614,Europa,False,E/603/S,TRAPPIST-1e,32.0,True,1003.0,909.0,0.0,0.0,15.0,False
8621,Europa,False,C/308/P,NaN,41.0,True,0.0,7964.0,0.0,3238.0,5839.0,False
8652,Europa,False,C/342/S,TRAPPIST-1e,36.0,True,0.0,5600.0,715.0,2868.0,971.0,True


In [384]:
def add_total(data):
#     vip_true_df = data[data['VIP'] == True]

# Calculate the total for each row
    total_values = data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)

# Add the total values as a new column
    data['Total'] = total_values

# Print the resulting DataFrame with the new 'Total' column
    return data


In [385]:
tot= add_total(space_train)
tot_fin= add_total(space_test)

In [386]:
tot.isnull().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
Total             0
dtype: int64

In [387]:
tot_fin.isnull().sum()

HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age               0
VIP              93
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Total             0
dtype: int64

In [388]:
def vip_set(df):
    # Set the 'VIP' feature to True for every null value where 'Total' variable is >= 4000
    df.loc[(df['Total'] >= 4000) & df['VIP'].isnull(), 'VIP'] = True
    
    # Set the 'VIP' feature to False for every value where 'Total' variable is < 4000
    df.loc[df['Total'] < 4000, 'VIP'] = False
    
    return df

In [389]:
tot=vip_set(tot)
tot_fin=vip_set(tot_fin)

In [395]:
tot.isnull().sum()
# tot_fin.isnull().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
Total             0
dtype: int64

In [396]:
tot_fin.isnull().sum()

HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Total             0
dtype: int64

In [397]:
def fill_destination_mode(df):
    mode_value = df['Destination'].mode()[0]  # Calculate the mode of the 'Destination' column
    df['Destination'].fillna(mode_value, inplace=True)  # Fill null values with the mode
    return df

In [398]:
tot=fill_destination_mode(tot)
tot_fin=fill_destination_mode(tot_fin)

/tmp/ipykernel_34/337189214.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Destination'].fillna(mode_value, inplace=True)  # Fill null values with the mode


In [399]:
tot.isnull().sum()
# tot_fin.isnull().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
Total             0
dtype: int64

In [400]:
tot_fin.isnull().sum()

HomePlanet       87
CryoSleep        93
Cabin           100
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Total             0
dtype: int64

In [401]:
def split_cabin_column(df):
    # Split the 'Cabin' column into three separate columns
    df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
    # Drop the original 'Cabin' column
    df.drop(columns=['Cabin'], inplace=True)
    return df

In [402]:
tot=split_cabin_column(tot)

In [403]:
tot_fin=split_cabin_column(tot_fin)

In [404]:
tot.isnull().sum()
# tot_fin.isnull().sum()

HomePlanet      201
CryoSleep       217
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
Total             0
Deck            199
Num             199
Side            199
dtype: int64

In [405]:
tot_fin.isnull().sum()

HomePlanet       87
CryoSleep        93
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Total             0
Deck            100
Num             100
Side            100
dtype: int64

In [341]:
tot.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Total,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0.0,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,736.0,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,10383.0,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,5176.0,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,1091.0,F,1,S


In [406]:
def fill_Deck_num_side(df):
    mode_value = df['Deck'].mode()[0]  
    df['Deck'].fillna(mode_value, inplace=True)  
    mode_value = df['Num'].mode()[0]  
    df['Num'].fillna(mode_value, inplace=True)  
    mode_value = df['Side'].mode()[0]  
    df['Side'].fillna(mode_value, inplace=True)  
    return df

In [343]:
tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8693 non-null   object 
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   object 
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
 10  Transported   8693 non-null   bool   
 11  Total         8693 non-null   float64
 12  Deck          8494 non-null   object 
 13  Num           8494 non-null   object 
 14  Side          8494 non-null   object 
dtypes: bool(1), float64(7), object(7)
memory usage: 959.4+ KB


In [344]:
tot.Deck.mode()

0    F
Name: Deck, dtype: object

In [407]:
tot=fill_Deck_num_side(tot)
tot_fin=fill_Deck_num_side(tot_fin)

/tmp/ipykernel_34/2988173421.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Deck'].fillna(mode_value, inplace=True)
/tmp/ipykernel_34/2988173421.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

In [408]:
tot.isnull().sum()

HomePlanet      201
CryoSleep       217
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
Total             0
Deck              0
Num               0
Side              0
dtype: int64

In [409]:
tot_fin.isnull().sum()

HomePlanet      87
CryoSleep       93
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Total            0
Deck             0
Num              0
Side             0
dtype: int64

In [413]:
mode_value = tot_fin['CryoSleep'].mode()[0]  
tot_fin['CryoSleep'].fillna(mode_value, inplace=True)  
mode_value = tot_fin['HomePlanet'].mode()[0]  
tot_fin['HomePlanet'].fillna(mode_value, inplace=True)  

/tmp/ipykernel_34/1874579318.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tot_fin['CryoSleep'].fillna(mode_value, inplace=True)


In [414]:
def drop_null_values(df):
    df.dropna(subset=['HomePlanet', 'CryoSleep'], inplace=True)
    return df

In [415]:
tot=drop_null_values(tot)

In [416]:
tot.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Total           0
Deck            0
Num             0
Side            0
dtype: int64

In [417]:
tot_fin.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Total           0
Deck            0
Num             0
Side            0
dtype: int64

In [418]:
tot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8277 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8277 non-null   object 
 1   CryoSleep     8277 non-null   object 
 2   Destination   8277 non-null   object 
 3   Age           8277 non-null   float64
 4   VIP           8277 non-null   object 
 5   RoomService   8277 non-null   float64
 6   FoodCourt     8277 non-null   float64
 7   ShoppingMall  8277 non-null   float64
 8   Spa           8277 non-null   float64
 9   VRDeck        8277 non-null   float64
 10  Transported   8277 non-null   bool   
 11  Total         8277 non-null   float64
 12  Deck          8277 non-null   object 
 13  Num           8277 non-null   object 
 14  Side          8277 non-null   object 
dtypes: bool(1), float64(7), object(7)
memory usage: 978.0+ KB


In [419]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [420]:
def preprocess_data(df):
    # Label encode the 'VIP' column
    label_encoder = LabelEncoder()
    df['VIP'] = label_encoder.fit_transform(df['VIP'])
    df['Deck'] = label_encoder.fit_transform(df['Deck'])
    df['Side'] = label_encoder.fit_transform(df['Side'])
    df['CryoSleep'] = label_encoder.fit_transform(df['CryoSleep'])
    # Perform one-hot encoding for categorical columns
    categorical_cols = ['HomePlanet', 'Destination']
    df = pd.get_dummies(df, columns=categorical_cols)
    
#     # Convert boolean column to integers
#     df['Transported'] = df['Transported'].astype(int)
    
    return df

In [421]:
tot=preprocess_data(tot)
tot_fin=preprocess_data(tot_fin)

In [422]:
tot.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Transported', 'Total', 'Deck', 'Num', 'Side',
       'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e'],
      dtype='object')

In [423]:
tot_fin.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Total', 'Deck', 'Num', 'Side', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e'],
      dtype='object')

In [424]:
tot.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Total,Deck,Num,Side,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,0.0,1,0,0,False,True,False,False,False,True
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,True,736.0,5,0,1,True,False,False,False,False,True
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,10383.0,0,0,1,False,True,False,False,False,True
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,5176.0,0,0,1,False,True,False,False,False,True
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,True,1091.0,5,1,1,True,False,False,False,False,True


In [425]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [426]:
y= tot['Transported']
X= tot.drop('Transported',axis=1)
X_train , X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [427]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_val)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


166/166 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.6456 - loss: 0.6216 - val_accuracy: 0.7713 - val_loss: 0.4570
Epoch 2/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7761 - loss: 0.4669 - val_accuracy: 0.7834 - val_loss: 0.4384
Epoch 3/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7781 - loss: 0.4601 - val_accuracy: 0.7721 - val_loss: 0.4337
Epoch 4/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7912 - loss: 0.4355 - val_accuracy: 0.7864 - val_loss: 0.4317
Epoch 5/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7961 - loss: 0.4284 - val_accuracy: 0.7849 - val_loss: 0.4243
Epoch 6/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7998 - loss: 0.4223 - val_accuracy: 0.7872 - val_loss: 0.4285
Epoch 7/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8036 - loss: 0.4205 - val_accuracy: 0.7887 - val_loss: 0.4254
Epoch 8/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7896 - loss: 0.4393 - val_accuracy: 0.7940 - val

In [428]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_val)
print(f"Test Accuracy: {test_accuracy}")

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7824 - loss: 0.4150
Test Accuracy: 0.7940821051597595


In [429]:
new_data_scaled = scaler.transform(tot_fin)
prediction = model.predict(new_data_scaled)


In [432]:
prediction

array([[0.5269136 ],
       [0.00578666],
       [0.9995325 ],
       ...,
       [0.6570545 ],
       [0.7656932 ],
       [0.6267207 ]], dtype=float32)

In [433]:
predicted_classes = (prediction > 0.5)

In [442]:
import numpy as np

predicted_classes = np.array(predicted_classes).flatten()

In [443]:
df = pd.DataFrame({"PassengerId": test_ids.values, "Transported": predicted_classes})

In [445]:
df.to_csv("submission.csv",index=False)